## 주식 가격 (대성에너지)

### 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [4]:
stock_code = '117580'   # 대성에너지 stock code

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 현재일-1일
prev_date = datetime.now() + relativedelta(days=-1)
end_date = prev_date.strftime("%Y%m%d")

In [2]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [5]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT tr_date FROM stock_price where stock_code = {stock_code} ORDER BY tr_date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

In [6]:
dt_last_date = datetime.strptime(last_date, '%Y%m%d')       # 날짜형으로 변환
dt_start_date = dt_last_date + relativedelta(days=+1)
start_date = dt_start_date.strftime("%Y%m%d")               # 문자형으로 변환

In [7]:
stock_code, last_date, start_date, end_date

('117580', '20230630', '20230701', '20230705')

#### 1. 주가 데이터 가져오기

In [8]:
from pykrx import stock

# 특정기간 특정 종목의 OHLCV
ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)

In [9]:
ohlcv.shape

(3, 7)

#### 2. 데이터 변환 및 전처리

In [10]:
df_ohlcv = ohlcv.reset_index()

In [11]:
ohlcv.shape

(3, 7)

In [12]:
df_ohlcv.dtypes

날짜      datetime64[ns]
시가               int32
고가               int32
저가               int32
종가               int32
거래량              int32
거래대금             int64
등락률            float32
dtype: object

In [13]:
# 날짜 형식 변환 => 문자열 yyyymmdd
df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')

In [14]:
df_ohlcv.dtypes

날짜       object
시가        int32
고가        int32
저가        int32
종가        int32
거래량       int32
거래대금      int64
등락률     float32
dtype: object

#### 3. DB 테이블에 저장

In [15]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_ohlcv.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                            values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
conn.commit()
conn.close()